<a href="https://colab.research.google.com/github/Aananda-giri/scripts/blob/main/mero_school_video_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# insatll and update ffmpeg
# source: https://launchpad.net/~savoury1/+archive/ubuntu/ffmpeg5
# !sudo add-apt-repository ppa:savoury1/ffmpeg4

# uncomment later
# !sudo add-apt-repository ppa:savoury1/ffmpeg5 # ppa to update ffmpeg to v5 default: v3
# !sudo apt-get update
# !sudo apt-get upgrade && sudo apt-get dist-upgrade
# !sudo apt-get install ffmpeg

# !sudo ppa-purge ppa:savoury1/ffmpeg4            # to uninstall ppa
# !apt --purge remove ffmpeg  # to uninstall ffmpeg



# installations
from google.colab import drive
drive.mount('/content/drive')

# updating nodejs from v14 -> v16 :: reference : https://www.digitalocean.com/community/tutorials/how-to-install-node-js-on-debian-10
# !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -
# !sudo apt install nodejs
# !node -v

# !apt-get update && upgrade
## Install google chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
# Install the package using the dpkg command:
!sudo dpkg -i google-chrome-stable_current_amd64.deb
# If you encounter any dependency errors during the installation, you can resolve them by running the following command:
!sudo apt-get install -f
# Verify the installation by running the following command:
!google-chrome --version


# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
# import sys
# sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
# !npm i convert-excel-to-json
!npm install puppeteer
!npm install puppeteer-extra
!npm install puppeteer-extra --save
!npm install puppeteer-extra-plugin-stealth --save   # installs plugin
!npm install dotenv --save
!npm install childprocess --save
# !npm install puppeteer-extra-plugin-recaptcha
# !npm install -g npm to update # update npm
# !npm install robots-txt-parser --save
# !npm install random-int




# -------------------------------------------------------------------------------------------------------
# installing ffmpeg:
# -------------------------------------------------------------------------------------------------------

# --------------------------
# Method IV :: https://www.ffmpeg.org/download.html#build-linux
# --------------------------
# https protocol not found, recompile FFmpeg with openssl, gnutls or securetransport enabled.
!sudo apt-get install libssl-dev  # to allow urls as input
!sudo apt-get install nasm        # ffmpeg seems to need it <nasm/yasm not found or too old. Use --disable-x86asm for a crippled build.>
!wget https://launchpad.net/ubuntu/+archive/primary/+sourcefiles/ffmpeg/7:5.1.2-3ubuntu1/ffmpeg_5.1.2.orig.tar.xz
!tar -xf ffmpeg_5.1.2.orig.tar.xz
!/bin/bash /content/ffmpeg-5.1.2/configure --enable-openssl

!make
!make install
!ffmpeg

# --------------------------
# Method III :: Not Working: https://www.ffmpeg.org/download.html#build-linux
# --------------------------
# !git clone https://git.ffmpeg.org/ffmpeg.git ffmpeg
# !/bin/bash /content/ffmpeg/configure  # 
# !sudo apt-get install yasm
# !make
# !make install
# !ffmpeg # ffmpeg version 4.2.7-0ubuntu0.1


# ------------------
# Method II: Not working: https://www.maskaravivek.com/post/how-to-install-ffmpeg-on-ec2-running-amazon-linux/
# ------------------
# !cd /usr/local/bin/ffmpeg
# !wget https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz  # download ffmpeg
# !tar -xf ffmpeg-release-amd64-static.tar.xz # extract it 
# !rm /usr/bin/ffmpeg # remove existing ffmpeg (v4.**)
# !ln -s /content/ffmpeg-6.0-amd64-static/ffmpeg /usr/bin/ffmpeg  # Create symbolic link

# ------------------------------
# Method I: Doesn't work anymore
# ------------------------------
# !sudo add-apt-repository -y ppa:mc3man/trusty-media
# !sudo apt-get update
# !sudo apt-get install --only-upgrade ffmpeg



In [ ]:
# stores code to file: 'soa_crawl.js'
%%writefile soa_crawl.js
'use strict';
// const excelToJson = require('convert-excel-to-json');
// const puppeteer = require('puppeteer');
const puppeteer = require('puppeteer-extra')
const dotenv = require('dotenv');
dotenv.config();

// add stealth plugin and use defaults (all evasion techniques)
const StealthPlugin = require('puppeteer-extra-plugin-stealth')
puppeteer.use(StealthPlugin())

var fs = require('fs'); // to create folder if not exist // reference: https://colab.research.google.com/drive/168X6Zo0Yk2fzEJ7WDfY9Q_0UOEmHSrZc?usp=sharing
const { exec } = require("childprocess");   // to execute linux-ffmpeg commands to download

// to wait
const delay = ms => new Promise(resolve => setTimeout(resolve, ms));


var root_dir = './drive/MyDrive/mero_school';  // <colab>
var browser_executable_path = '/opt/brave.com/brave/brave-browser'
// browser_executable_path =  '/opt/google/chrome/google-chrome'

if (!fs.existsSync('./drive/MyDrive')){
  // credential for running locally
  root_dir = 'mero_school' 
  browser_executable_path = "/usr/bin/chromium-browser";  
  // browser_executable_path = "/usr/bin/chromedriver";  
}

// MeroSchool credentials
// const CREDS = { username: '9828905351', password: 'fluidmechanics' };
const CREDS = { username: '9868960200', password: 'redminote7pro' };


/*
// folders path for each download link
const subjectFolderName =  `${courseName}[${grade}][${instructorName}]`;
const downloadPath = rootDir + subjectFolderName;
*/

function createFolder (folder_name){
  /*
  Creates folder is doesnot already esists
  */
    if ((folder_name != root_dir)) {
        if (!fs.existsSync(root_dir + '/' + folder_name)){
            if (root_dir != ''){
                fs.mkdirSync(root_dir + '/' + folder_name);
            } else {
                fs.mkdirSync(folder_name);
            }
            // console.log(`created folder: ${root_dir + '/' + folder_name}`);
        } else {
            // console.log(`folder: ${root_dir + '/' + folder_name} exists`);
        }
    } else{
        createFolder('');   // trying create root folder
    }
}
createFolder(root_dir)

// js progress generator function
function progress_bar(current_progress, total, label){
  let progress = Math.round((current_progress/total)*100);
  console.log(`${label}_progress  : ${progress}% :: `, current_progress, '/', total);
  console.log(total);
  let bar = [];
  for (let i = 0; i < 100; i++){
    if (i < progress){
      bar.push('█');
    } else {
      bar.push('░');
    }
  }
  // return bar.join(''); // array to string
  console.log(bar.join(''));
}


// create file 'm3u8_urls.json' if file doesnot exist
if (!fs.existsSync(root_dir + 'm3u8_urls.json')){
  console.log('\ncreating file: m3u8_urls.json...\n');
  fs.writeFileSync(root_dir + 'm3u8_urls.json', JSON.stringify(
    {
      "comment":"progress for sheets are generated and updated based on sheet_name automatically .."
    }
  ));
 }


function load_json_data(file_path){
  // loading error links
  // var saved_data;
  try {
      var saved_data = fs.readFileSync(root_dir + '/' + file_path, 'utf-8');
      saved_data = JSON.parse(saved_data);
      // console.log('Loaded Links...: \n ' + saved_data);
      return saved_data;
  } catch (error) {
      console.log('Error Loading json file ...: \n ' + root_dir + '/' + file_path + error); 
  }
}

function save_json_data(data, file_path){
  let to_save_data  = JSON.stringify(data);
  try {
    fs.writeFileSync(root_dir + '/' + file_path, to_save_data);
  } catch (error) {
    console.log('Error saving data to file error_data...' + root_dir + '/' + file_path + error);
  }
}

var myCourses = []; //'https://mero.school/videoplay/97/19252' ];
var videoUrls = [];
var m3u8_urls = [];


(async () => {
// ----------------------------------------------------------------------------
// --------------------------------------  Crawl and save urls  ----------------
// ----------------------------------------------------------------------------

  /*
  const browser = await puppeteer.launch({
    headless:false, ignoreHTTPSErrors: true,
    defaultViewport: null,
    // args: ['--start-fullscreen'],
    executablePath: browser_executable_path
    
  });*/
  
  // const browser = await puppeteer.launch({executablePath: '/opt/google/chrome/chrome', args:['--no-sandbox','--start-maximized'], ignoreHTTPSErrors: true, headless: true}); // aws
  const browser = await puppeteer.launch({executablePath: "/opt/google/chrome/chrome", args:['--no-sandbox','--start-maximized']}); // colab
  const page = await browser.newPage();
  await page.setExtraHTTPHeaders({
      'Accept-Language': 'en-US,en;q=0.9',
    });
    
    // login
    // async function login(){
      await page.goto('https://mero.school/login', {timeout: 0});
      console.log('login page loaded');
      // await delay(6000);
      // await page.waitForNavigation({waitUntil: 'networkidle2'})
      
      await page.type('#phoneno', CREDS.username);
      // document.querySelector('input[type=password]')
      
      await page.type('input[type=password]', CREDS.password);
      await page.click('.btn-full-width');
    // }

    console.log('logging in')
    await page.waitForNavigation({
      // waitUntil: 'networkidle0',
    });
    console.log('wait over')
    
   // accepts confirmation alert
   page.on('dialog', async dialog => {
      //get alert message
      console.log(dialog.message());
      //accept alert
      await dialog.accept();
      console.log('accept');
   })


    try{
      // try to click on 'logout everywhere' button if it exists
        await page.click('.btn-search-gs');    
        console.log('multiple login')
    } catch(e){
        console.log('Not an Error: ' + e);
    }
    
    
    
    
    let results = [];
    // my_courses
    if (myCourses.length < 1){
      await page.goto('https://mero.school/my_courses', {timeout: 0}); // got myCourses

      // list of myCourses links
      results = await page.evaluate(() => {
        let res = []
        for (let u of document.querySelectorAll('a')) {if (u.href.startsWith('https://mero.school/videoplay/')){res.push(u.href)}}
        res= [...new Set(res)]
        return res;
      });

        
      // for (u of page.$$('a')) {if (u.href.startsWith('https://mero.school/videoplay/')){results.push(u.href)}}
          
      // myCourses += results
      Array.prototype.push.apply(myCourses, results);
      console.log('\n\n myCourses: ', myCourses);
    }


    
    // async function crawl_video_urls(){
    for (let course_url of myCourses){
      
      await page.goto(course_url, {timeout: 0});
      
      
      
      
      results  = await page.evaluate(() =>{
        let sub_id = window.location.href.split('/')[4];
        console.log(sub_id);
        let res = []
        for (let vid_id of document.querySelectorAll('li [type=checkbox]')){
          res.push('https://mero.school/videoplay/' + sub_id + '/' + vid_id.id)
          // console.log(i.id);
        }
        res= [...new Set(res)]
        return res;
      })
      // videoUrls += results
      Array.prototype.push.apply(videoUrls, results);
    }
    console.log('\n\nvid_urls: ', videoUrls)
    














    // results = [];  // collects all m33u8 urls
    // let paused = false;
    // let pausedRequests = [];

    // const nextRequest = () => { // continue the next request or "unpause"
    //     if (pausedRequests.length === 0) {
    //         paused = false;
    //     } else {
    //         // continue first request in "queue"
    //         (pausedRequests.shift())(); // calls the request.continue function
    //     }
    // };

    // await page.setRequestInterception(true);
    // page.on('request', request => {
    //     if (paused) {
    //         pausedRequests.push(() => request.continue());
    //     } else {
    //         paused = true; // pause, as we are processing a request now
    //         request.continue();
    //     }
    // });

    page.on('requestfinished', async (request) => {
        const response = await request.response();

        const responseHeaders = response.headers();
        // let responseBody;
        // if (request.redirectChain().length === 0) {
        //     // body can only be access for non-redirect responses
        //     responseBody = await response.buffer();
        // }

        // const information = {
        //     url: request.url(),
            // requestHeaders: request.headers(),
            // requestPostData: request.postData(),
            // responseHeaders: responseHeaders,
            // responseSize: responseHeaders['content-length'],
            // responseBody,
        // };
        
        if (request.url().includes('https://originvideo.mero.school/') && request.url().endsWith('.m3u8')) {
          if (results.indexOf(request.url()) == -1){
            results.push(request.url());
            console.log('returning', request.url());
            } else {
            console.log('url already exists', request.url());
            }

        }
        
            // nextRequest(); // continue with next request
    });
    page.on('requestfailed', (request) => {
        // handle failed request
        // nextRequest();
        // pass
    });
    
    


    
    
    // test :: no. of <li> urls = no. of downloaded files





    // async function crawl_m3u8_urls(){
      results = []
    // for (let vid_url of videoUrls.slice(1,4)){
    for (let [index, vid_url] of videoUrls.entries()){
      await delay(4000);
      progress_bar(index, videoUrls.length, 'crawling_m3u8');
      console.log('results',results, '\n\n')
      
      console.log('\nopening video: ', vid_url);
      // let page = await browser.newPage();
      // page.close()
      await save_json_data(results, 'm3u8_urls.json')
      await page.goto(vid_url, { timeout: 0 });
      
      
      
    }
    // console.log(m3u8_urls);
    results = [...new Set(results)]
    
    // Logout before closing browser
    await page.goto("https://mero.school/login/logout", { timeout: 0 });
    await page.close()
    await browser.close();
    // Array.prototype.push.apply(m3u8_urls, results);

    // login();
    // crawl_my_courses();


// */
/*
// ----------------------------------------------------------------------------
// -------------------------------------- clean urls and download ----------------
// ----------------------------------------------------------------------------

links = load_json_data('m3u8_urls.json');

for (let link of links){
  
  // console.log(link)
  if (!(link.includes('master.m3u8'))){
    // console.log(link);
    let file_name = link.split('/')[4];
    m3u8_master.push(link.split(file_name)[0] + file_name +'/v3/prog_index.m3u8');
  } else {
    m3u8_master.push(link);
  }
}
// m3u8_master = [... new Set(m3u8_master)]
// save_json_data(m3u8_master, 'm3u8_master.json')

// download file:: name: filename, location : subject_array_folders
// execute linux commands from node
// source: https://stackabuse.com/executing-shell-commands-with-node-js/
// decodeURI : https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/decodeURI
// download_command: ffmpeg -i "https://originvideo.mero.school/physics%2012/Unit-1.%20Mechanics%7CChapter%201-%20Rotational%20Dynamics%7C1.%20Rigid%20Bodies-Rb7Rd/master.m3u8" -codec copy file.mp4
// source: https://stackoverflow.com/questions/47233304/how-to-download-m3u8-in-once-time

// links = load_json_data('m3u8_master.json');
links = [... new Set(m3u8_master)]
try{
 for (let [link_count, link] of links.slice(0,45).entries()){
    let subject_name = decodeURI(link.split('/')[3]);
  let file_name = root_dir + '/' + subject_name + '/' + decodeURI(link.split('/')[4]) +'.mp4';
  console.log(`\n\nfile_name:${file_name}\n\n`)
  createFolder(subject_name);
  
  try{
    await exec(`ffmpeg -i ${link} -codec copy \"${file_name}\"`, (error, stdout, stderr) => {
            if (error) {
                console.log(`error: ${error.message}`);
                return;
            }
            if (stderr) {
                console.log(`stderr: ${stderr}`);
                return;
            }
            console.log(`stdout: ${stdout}`);
        });
      
      progress_bar(link_count, links.length, 'download');
      //delay(15000);
  } catch(error){
    console.log('err. inner : ', error)
  }
 } } catch(error){
    console.log('err. outer : ', error)
  }



// add download root and make workable with array

console.log('\n\n --------------- ************************* --------------- ');
console.log(' --- completed ffmpeg code btn -> download in progress --- \n ------------ waiting 10 minutes ------------ ');
console.log(' --------------- ************************* --------------- \n\n ');
await delay(3000000); // waiting 50 minutes before closing browser after clicking download to all links of specific sheet





// ----------------------------------------------------------------------------
// --------------------------------------  ----------------
// ----------------------------------------------------------------------------

*/
})();

/*
async function login(){}
*/

Writing soa_crawl.js


In [ ]:
!node soa_crawl.js

# Download urls


In [ ]:
!ffmpeg -i "https://originvideo.mero.school/electronic%20devices%20and%20circuits%20%28iii_i%29/1.%20Marks%20distribution%7CIntroduction-qckjB/master.m3u8" -c copy '/content/drive/MyDrive/mero_school/fuck_ffmpeg.mp4'
# !mkdir ffmpeg && cd ffmpeg
# !wget https://launchpad.net/ubuntu/+archive/primary/+sourcefiles/ffmpeg/7:5.1.2-3ubuntu1/ffmpeg_5.1.2.orig.tar.xz
# !ls
# !unzip
# !apt-get install libpostproc

# Testing ffmpeg

In [ ]:
# !rm /bin/bash/ffmpeg
# !git clone https://git.ffmpeg.org/ffmpeg.git ffmpeg
!ffmpeg0/configure

# !ffmpeg
# !ffmpeg -i "https://originvideo.mero.school/electromagnetics/Question%20Bank%7C2076%20Chaitra%7CQuestion%20No.%2011-DEURI/master.m3u8" -codec copy '/content/drive/MyDrive/mero_school/test2.mp4'
# !sudo apt install ffmpeg
# !ffmpeg
# !ffmpeg -i https://demo.unified-streaming.com/k8s/features/stable/video/tears-of-steel/tears-of-steel.ism/.m3u8 -c:v libx264 -preset slow -crf 22 "saveas.mp4"
# ffmpeg -i https://originvideo.mero.school/electromagnetics/0.%20Marks%20Distribution%7CSyllabus%20and%20Marks%20Distribution-AIb7B/v3/prog_index.m3u8 -codec copy "/content/drive/MyDrive/mero_school/electromagnetics/0. Marks Distribution|Syllabus and Marks Distribution-AIb7B.mp4"

# !sudo add-apt-repository --remove ppa:mc3man/trusty-media
# !sudo add-apt-repository ppa:savoury1/ffmpeg4
# !sudo apt update
# !sudo apt install ffmpeg

# !sudo apt --fix-broken install


nasm/yasm not found or too old. Use --disable-x86asm for a crippled build.

If you think configure made a mistake, make sure you are using the latest
version from Git.  If the latest version fails, report the problem to the
ffmpeg-user@ffmpeg.org mailing list or IRC #ffmpeg on irc.libera.chat.
Include the log file "ffbuild/config.log" produced by configure as this will help
solve the problem.


# Python code to download
  * Works

In [ ]:
!pip install ffmpeg-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import ffmpeg, subprocess
import concurrent.futures
import json, urllib, os
root_dir = "/content/drive/MyDrive/mero_school/"

def get_urls():
  with open(root_dir + "m3u8_urls.json","r") as f:
    urls = json.load(f)
  urls = list(set([url.replace('/v0/prog_index.m3u8', '/master.m3u8').replace('/v2/prog_index.m3u8', '/master.m3u8').replace('/v1/prog_index.m3u8', '/master.m3u8') for url in urls]))
  print(f'\n\n urls: {len(urls)}')
  return urls

def download_video(url):
    
    url_decoded = urllib.parse.unquote_plus(url)
    subject_name = url_decoded.split('/')[3]
    if subject_name not in os.listdir(root_dir):
      os.mkdir(root_dir + subject_name)
    # print(f'root:{root_dir}\n\n sub:{subject_name}\n\nlist:{os.listdir(root_dir)}')
    path = root_dir + subject_name + '/' + url_decoded.split('/')[4] + '.mp4'
    
    # download if does not exist
    if not os.path.exists(path):
      print(f'\n\npath:{path}')
      command = [
        "ffmpeg",
        "-i", url,
        "-codec", "copy",
        path
      ]

      process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
      stdout, stderr = process.communicate()

      if process.returncode != 0:
        print(f"Error: {stderr.decode()}")

# url = "https://originvideo.mero.school/electronic%20devices%20and%20circuits%20%28iii_i%29/1.%20Marks%20distribution%7CIntroduction-qckjB/v2/prog_index.m3u8"
# path = "test.mp4"
# download_video(url)
# Use ThreadPoolExecutor to download the videos concurrently

# for n,url in enumerate(urls):
'''with concurrent.features.ThreadPoolExecutor(max_workers=15) as executor:
    # print(url, path)
    # print(n)
    # download_video(url, path)
    # Submit tasks to the executor
    futures = [executor.submit(download_video, url) for url in urls]
    # Wait for all tasks to complete
    concurrent.futures.wait(futures)'''
urls = get_urls()
with concurrent.futures.ThreadPoolExecutor(max_workers=50) as executor:
    # Submit tasks to the executor
    futures = [executor.submit(download_video, url) for url in urls]
    # Wait for all tasks to complete
    concurrent.futures.wait(futures)

n=0
n+=len(os.listdir(root_dir+ 'basic electrical engineering/'))
n+=len(os.listdir(root_dir+'electromagnetics/'))
n+=len(os.listdir(root_dir+'electronic devices and circuits (iii_i)/'))
n



 urls: 547


547

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os, json
root_dir = "/content/drive/MyDrive/mero_school/"

def get_urls():
  with open(root_dir + "m3u8_urls.json","r") as f:
    urls = json.load(f)
  urls = list(set([url.replace('/v0/prog_index.m3u8', '/master.m3u8').replace('/v2/prog_index.m3u8', '/master.m3u8').replace('/v1/prog_index.m3u8', '/master.m3u8') for url in urls]))
  print(f'\n\n urls: {len(urls)}')
  return urls
a=get_urls()

n=0
n+=len(os.listdir(root_dir+ 'basic electrical engineering/'))
n+=len(os.listdir(root_dir+'electromagnetics/'))
n+=len(os.listdir(root_dir+'electronic devices and circuits (iii_i)/'))
n

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


 urls: 547


547

## copy files
  * iterate through all the folders in source dir
  * if file doesnt exist in destination: move file from source to destination 
  * if file exist in destination: move file if size in destination is less than size in source

In [ ]:
import os
import shutil


root_dir = "/content/drive/MyDrive/mero_school/"
source_dirs = ['basic electrical engineering0', 'electromagnetics0', 'electronic devices and circuits (iii_i)0']
destination_dirs = ['basic electrical engineering', 'electromagnetics', 'electronic devices and circuits (iii_i)']


def move_if_not_exist_and_keep_larger_file(source, destination):
    source_dir = os.path.join(root_dir, source)
    dest_dir = os.path.join(root_dir, destination)

    for file_name in os.listdir(source_dir):
        src_file_path = os.path.join(source_dir, file_name)
        dest_file_path = os.path.join(dest_dir, file_name)

        if not os.path.exists(dest_file_path):
            shutil.move(src_file_path, dest_file_path)
            print(f"Moved {file_name} to {dest_dir}")
        else:
            src_file_size = os.path.getsize(src_file_path)
            dest_file_size = os.path.getsize(dest_file_path)
            if src_file_size > dest_file_size:
                os.remove(dest_file_path)
                shutil.move(src_file_path, dest_file_path)
                print(f"Replaced {file_name} in {dest_dir} with larger file")
            else:
                os.remove(src_file_path)
                print(f"Ignored {file_name} as it already exists in {dest_dir} and is larger")

    print("Done!")

for source, destination in zip(source_dirs, destination_dirs):
    print(f"\'{str(source)}\', \'{str(destination)}\'")
    move_if_not_exist_and_keep_larger_file(source, destination)

# JS code to download
* Have to give root_dir mannually
* works?

In [ ]:
%%writefile save_ffmpeg_code.js
var fs = require('fs');
const { exec } = require("child_process");	// to execute linux-ffmpeg commands to download
var root_dir = '/content/drive/MyDrive/mero_school';  // <colab>

function load_json_data(file_path){
  // loading error links
  // var saved_data;
  try {
      var saved_data = fs.readFileSync(root_dir + '/' + file_path, 'utf-8');
      saved_data = JSON.parse(saved_data);
      // console.log('Loaded Links...: \n ' + saved_data);
      return saved_data;
  } catch (error) {
      console.log('Error Loading json file ...: \n ' + root_dir + '/' + file_path + error); 
  }
}


function save_json_data(data, file_path){
  let to_save_data  = JSON.stringify(data);
  try {
    fs.writeFileSync(root_dir + '/' + file_path, to_save_data);
  } catch (error) {
    console.log('Error saving data to file error_data...' + root_dir + '/' + file_path + error);
  }
}

function createFolder (folder_name){
  /*
  Creates folder is doesnot already esists
  */
	if ((folder_name != root_dir)) {
		if (!fs.existsSync(root_dir + '/' + folder_name)){
			if (root_dir != ''){
				fs.mkdirSync(root_dir + '/' + folder_name);
			} else {
				fs.mkdirSync(folder_name);
			}
			// console.log(`created folder: ${root_dir + '/' + folder_name}`);
		} else {
			// console.log(`folder: ${root_dir + '/' + folder_name} exists`);
		}
	} else{
		createFolder('');	// trying create root folder
	}
}

// js progress generator function
function progress_bar(current_progress, total, label){
  let progress = Math.round((current_progress/total)*100);
  console.log(`${label}_progress  : ${progress}% :: `, current_progress, '/', total);
  console.log(total);
  let bar = [];
  for (let i = 0; i < 100; i++){
    if (i < progress){
      bar.push('█');
    } else {
      bar.push('░');
    }
  }
  // return bar.join(''); // array to string
  console.log(bar.join(''));
}


createFolder(root_dir)


var links = load_json_data('m3u8_urls.json');
console.log('linsk:',links)
// let links = ["https://originvideo.mero.school/hydraulics/1.%20Marks%20Distribution%7CSyllabus%20and%20Marks%20distribution-8zRyL/v2/prog_index.m3u8","https://originvideo.mero.school/hydraulics/1.%20Marks%20Distribution%7CSyllabus%20and%20Marks%20distribution-8zRyL/master.m3u8"]



let m3u8_master = []

// pre-processing
for (let link of links){
  
  console.log(link)
  if (!(link.includes('master.m3u8'))){
    // console.log(link);
    let file_name = link.split('/')[4];
    m3u8_master.push(link.split(file_name)[0] + file_name +'/v3/prog_index.m3u8');
  } else {
    m3u8_master.push(link);
  }
}

var ffmpeg_code = []
links = [... new Set(m3u8_master)]
 for (let [link_count, link] of links.slice(0,45).entries()){
	let subject_name = decodeURI(link.split('/')[3]);
  let file_name = root_dir + '/' + subject_name + '/' + decodeURI(link.split('/')[4]) +'.mp4';
  console.log(`\n\nfile_name:${file_name}\n\n`)
  createFolder(subject_name);
  ffmpeg_code.push(`/usr/bin/ffmpeg -i ${link} -codec copy \"${file_name}\"`);
  // console.log(`ffmpeg -i ${link} -codec copy \"${file_name}\"`);
 }

 save_json_data(ffmpeg_code, "ffmpeg_code.json")


In [ ]:
!node save_ffmpeg_code.js
# !ls /content/drive/MyDrive/mero_school/Hydraulics/ffmpeg_code.json

## SSH code to download
* Have to give root_dir mannually
* Works?

In [ ]:
%%writefile ssh_loop.sh
#!/bin/bash

# Read the JSON file into a variable
list=$(cat "/content/drive/MyDrive/mero_school/ffmpeg_code.json" | sed -E 's/\[(.*)\]/\1/' | sed -E 's/"//g')

# Split the list into individual commands
IFS=',' read -ra cmds <<< "$list"

# Use for loop to iterate over the commands
for cmd in "${cmds[@]}"; do
  # Execute each command
  echo $cmd
  # eval $cmd
done




In [ ]:
# %%writefile test.json
# ["hi", "hello"]
# %%writefile file.json
# ["echo hi", "echo hello"]

!chmod +x ./ssh_loop.sh
!./ssh_loop.sh
# !cp /content/file.json /content/drive/MyDrive/mero_school/Hydraulics/file.json

In [ ]:
ffmpeg -i https://originvideo.mero.school/electromagnetics/0.%20Marks%20Distribution%7CSyllabus%20and%20Marks%20Distribution-AIb7B/v3/prog_index.m3u8 -codec copy "/content/drive/MyDrive/mero_school/electromagnetics/0. Marks Distribution|Syllabus and Marks Distribution-AIb7B.mp4"

In [ ]:
!ffmpeg -v

In [ ]:
# !ls /content/drive/MyDrive/mero_school/Hydraulics

In [ ]:
# a="https://originvideo.mero.school/hydraulics"
# a.split('/')[3]

import urllib.parse
encodedStr = "https://originvideo.mero.school/hydraulics/1.%20Marks%20Distribution%7CSyllabus%20and%20Marks%20distribution-8zRyL/v1/prog_index.m3u8"
urllib.parse.unquote_plus(encodedStr).split('/')[4] + '.mp4'

## Download url from js
* Works?

In [ ]:
%%writefile save_videos.js
var fs = require('fs');
const { exec } = require("child_process");	// to execute linux-ffmpeg commands to download
var root_dir = '/content/drive/MyDrive/mero_school';  // <colab>

function load_json_data(file_path){
  // loading error links
  // var saved_data;
  try {
      var saved_data = fs.readFileSync(root_dir + '/' + file_path, 'utf-8');
      saved_data = JSON.parse(saved_data);
      // console.log('Loaded Links...: \n ' + saved_data);
      return saved_data;
  } catch (error) {
      console.log('Error Loading json file ...: \n ' + root_dir + '/' + file_path + error); 
  }
}

function createFolder (folder_name){
  /*
  Creates folder is doesnot already esists
  */
	if ((folder_name != root_dir)) {
		if (!fs.existsSync(root_dir + '/' + folder_name)){
			if (root_dir != ''){
				fs.mkdirSync(root_dir + '/' + folder_name);
			} else {
				fs.mkdirSync(folder_name);
			}
			// console.log(`created folder: ${root_dir + '/' + folder_name}`);
		} else {
			// console.log(`folder: ${root_dir + '/' + folder_name} exists`);
		}
	} else{
		createFolder('');	// trying create root folder
	}
}

// js progress generator function
function progress_bar(current_progress, total, label){
  let progress = Math.round((current_progress/total)*100);
  console.log(`${label}_progress  : ${progress}% :: `, current_progress, '/', total);
  console.log(total);
  let bar = [];
  for (let i = 0; i < 100; i++){
    if (i < progress){
      bar.push('█');
    } else {
      bar.push('░');
    }
  }
  // return bar.join(''); // array to string
  console.log(bar.join(''));
}


createFolder(root_dir)


links = load_json_data('m3u8_urls.json');
// let links = ["https://originvideo.mero.school/hydraulics/1.%20Marks%20Distribution%7CSyllabus%20and%20Marks%20distribution-8zRyL/v2/prog_index.m3u8","https://originvideo.mero.school/hydraulics/1.%20Marks%20Distribution%7CSyllabus%20and%20Marks%20distribution-8zRyL/master.m3u8"]








let m3u8_master = []

// pre-processing
for (let link of links){
  
  console.log(link)
  if (!(link.includes('master.m3u8'))){
    // console.log(link);
    let file_name = link.split('/')[4];
    m3u8_master.push(link.split(file_name)[0] + file_name +'/v3/prog_index.m3u8');
  } else {
    m3u8_master.push(link);
  }
}
// m3u8_master = [... new Set(m3u8_master)]
// save_json_data(m3u8_master, 'm3u8_master.json')

// download file:: name: filename, location : subject_array_folders
// execute linux commands from node
// source: https://stackabuse.com/executing-shell-commands-with-node-js/
// decodeURI : https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/decodeURI
// download_command: ffmpeg -i "https://originvideo.mero.school/physics%2012/Unit-1.%20Mechanics%7CChapter%201-%20Rotational%20Dynamics%7C1.%20Rigid%20Bodies-Rb7Rd/master.m3u8" -codec copy file.mp4
// source: https://stackoverflow.com/questions/47233304/how-to-download-m3u8-in-once-time

// links = load_json_data('m3u8_master.json');
links = [... new Set(m3u8_master)]
//try{
 for (let [link_count, link] of links.slice(0,45).entries()){
	let subject_name = decodeURI(link.split('/')[3]);
  let file_name = root_dir + '/' + subject_name + '/' + decodeURI(link.split('/')[4]) +'.mp4';
  console.log(`\n\nfile_name:${file_name}\n\n`)
  createFolder(subject_name);
  
  console.log(`ffmpeg -i ${link} -codec copy \"${file_name}\"`);

  //try{
	exec(`ffmpeg -i ${link} -codec copy \"${file_name}\"`, (error, stdout, stderr) => {
			if (error) {
				console.log(`error: ${error.message}`);
				return;
			}
			if (stderr) {
				console.log(`stderr: ${stderr}`);
				return;
			}
			console.log(`stdout: ${stdout}`);
  		});
      
      progress_bar(link_count, links.length, 'download');
      //delay(15000);
// } catch(error){
//    console.log('err. inner : ', error)
//  }
 } 
// } catch(error){
//    console.log('err. outer : ', error)
//  }



// add download root and make workable with array

console.log('\n\n --------------- ************************* --------------- ');
console.log(' --- completed ffmpeg code btn -> download in progress --- \n ------------ waiting 10 minutes ------------ ');
console.log(' --------------- ************************* --------------- \n\n ');
//delay(3000000); // waiting 50 minutes before closing browser after clicking download to all links of specific sheet


(function () {
  //console.log('hi', links);
  for (let i of [1,2,3]){
    console.log(i)
    }
})();



In [ ]:
# !ls /content/drive/MyDrive/mero_school/Hydraulics/m3u8_urls.json
!node save_videos.js
# !ls /content/drive/MyDrive/mero_school


In [ ]:
!ffmpeg -i "https://originvideo.mero.school/hydraulics/Question%20Bank%7C2078%20Baisakh%7C10.%20Q.%20No.%205.a-RFHux/v2/prog_index.m3u8" -codec copy "test.mp4"

In [ ]:
for

# Test

In [ ]:
%%writefile test.js
const puppeteer = require('puppeteer');
//varv');
const dotenv = require('dotenv');
dotenv.config();

(async () => {
  //const browser = await puppeteer.launch({executablePath: process.env.browser_executable_path});
  // const browser = await puppeteer.launch({executablePath: "/usr/bin/chromium-browser"});
  const browser = await puppeteer.launch({executablePath: "/opt/google/chrome/chrome", args:['--no-sandbox','--start-maximized']});
  // const browser = await puppeteer.launch({executablePath: "/opt/brave.com/brave", headless: true});
  const page = await browser.newPage();

  await page.goto('https://www.google.com');

  await page.screenshot({
    path: 'google.png',
    fullPage: true
  });

  await browser.close();
})();

# chromium not working on google colab
Hi, I want to run puppeteer on google colab.

## running test code with `!node --trace-warnings test.js` says:
```
Command '/usr/bin/chromium-browser' requires the chromium snap to be installed. Please install it with:snap install chromium
```

## installed chromium with

```
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

!apt install chromium-browser  # also gave same result
```

## puppeteer code
```
const puppeteer = require('puppeteer');
//varv');
const dotenv = require('dotenv');
dotenv.config();

(async () => {
  const browser = await puppeteer.launch({executablePath: '/usr/bin/chromium-browser', args:['--no-sandbox','--start-maximized']});
  const page = await browser.newPage();

  await page.goto('https://www.google.com');

  await page.screenshot({
    path: 'google.png',
    fullPage: true
  });

  await browser.close();
})();
```
## Tried
* I tried installing chromium with 'snap install chromium' it says: 
error: cannot communicate with server: Post http://localhost/v2/snaps/chromium: dial unix /run/snapd.socket: connect: no such file or directory

* Tried installing snapd<br>
* `!systemctl status snapd.service`  # snapd: unrecognized service
* `!sudo apt update && upgrade`
* `!sudo apt install snapd`<br>
* `!which snapd`    # error: cannot communicate with server: Post http://localhost/v2/snaps/chromium: dial unix /run/snapd.socket: connect: no such file or directory

## Tried brave-browser
* installation code: https://brave.com/linux/
* executablePath: `"/opt/brave.com/brave"`
* error: `(node:42514) UnhandledPromiseRejectionWarning: Error: Failed to launch the browser process! spawn /opt/brave.com/brave EACCES`
* `chmod +x /opt/brave.com/brave` # did not solve

## Tried google-chrome: **Worked**
* executablePath: `"/opt/google/chrome/chrome"`
* installation code: 

* Add the Google Chrome repository to your system: <br>
`wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb`

* Install the package using the dpkg command:<br>
`sudo dpkg -i google-chrome-stable_current_amd64.deb`

* If you encounter any dependency errors during the installation, you can resolve them by running the following command:<br>
`sudo apt-get install -f`

* Verify the installation by running the following command:<br>
`google-chrome --version`






## **references**:<br>
https://colab.research.google.com/drive/168X6Zo0Yk2fzEJ7WDfY9Q_0UOEmHSrZc?usp=sharing#scrollTo=_Yf4OfPBAAPR<br><br>
https://forum.snapcraft.io/t/snap-d-error-cannot-communicate-with-server-connection-refused/6093/23

